In [96]:
%matplotlib inline

import os
import shutil
import time

from IPython.display import Image
import matplotlib.pyplot as plt

import numpy as np
import scipy
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import DataLoader

In [97]:
# Dataset Parameters
batch_size = 200
load_size = 128
fine_size = 128
c = 3
data_mean = np.asarray([0.45834960097,0.44674252445,0.41352266842])

use_cuda = True

model_path = '../../data/models/resnet50_ep2.pt'

In [98]:
def construct_dataloader_test():
    # Construct DataLoader
    opt_data_test = {
        'data_root': '../../data/images/',
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean
    }
    
    return DataLoader.DataLoaderDiskTest(**opt_data_test)

In [99]:
def test(test_loader, model):
    # switch to evaluate mode
    model.eval()

    preds = []
    
    for i in range(test_loader.size() // batch_size):
        input, paths = test_loader.next_batch(batch_size)
        if use_cuda:
            input = input.cuda(async=True)
        input_var = torch.autograd.Variable(input, volatile=True)

        # compute output
        output = model(input_var)
        preds.append((paths, output))

        print('Test: [{0}/{1}]\t'.format((i+1)*batch_size, test_loader.size()))

    return preds

In [100]:
model = torch.load(model_path)
model.cuda()

ResNet (
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU (inplace)
  (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential (
    (0): Bottleneck (
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (downsample): Sequential (
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (1): Bott

In [103]:
test_loader = construct_dataloader_test()
probs = test(test_loader, model)

Test: [200/10000]	
Test: [400/10000]	
Test: [600/10000]	
Test: [800/10000]	
Test: [1000/10000]	
Test: [1200/10000]	
Test: [1400/10000]	
Test: [1600/10000]	
Test: [1800/10000]	
Test: [2000/10000]	
Test: [2200/10000]	
Test: [2400/10000]	
Test: [2600/10000]	
Test: [2800/10000]	
Test: [3000/10000]	
Test: [3200/10000]	
Test: [3400/10000]	
Test: [3600/10000]	
Test: [3800/10000]	
Test: [4000/10000]	
Test: [4200/10000]	
Test: [4400/10000]	
Test: [4600/10000]	
Test: [4800/10000]	
Test: [5000/10000]	
Test: [5200/10000]	
Test: [5400/10000]	
Test: [5600/10000]	
Test: [5800/10000]	
Test: [6000/10000]	
Test: [6200/10000]	
Test: [6400/10000]	
Test: [6600/10000]	
Test: [6800/10000]	
Test: [7000/10000]	
Test: [7200/10000]	
Test: [7400/10000]	
Test: [7600/10000]	
Test: [7800/10000]	
Test: [8000/10000]	
Test: [8200/10000]	
Test: [8400/10000]	
Test: [8600/10000]	
Test: [8800/10000]	
Test: [9000/10000]	
Test: [9200/10000]	
Test: [9400/10000]	
Test: [9600/10000]	
Test: [9800/10000]	
Test: [10000/10000]	


In [119]:
with open(model_path + '_preds_fixed.txt', 'w') as f:
    for paths, batch_probs in probs:
        batch_preds = batch_probs.data.topk(5, 1, True, True)[1]
        for path, top5 in zip(paths, batch_preds):
            path = 'test/' + os.path.basename(path)
            f.write('{} {} {} {} {} {}\n'.format(path, *top5))